## Exploding Arrays into Records
Let us understand how to deal with array type columns in the Data Frame.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Special Data Types'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [ ]:
employees = [
     (2, "Henry", "Ford", 1250.0, 
      "India", ['henry@ford.com', 'hford@companyx.com'], 
      {"Home": "+91 234 567 8901", "Office": "+91 345 678 9012"}, 
      "456 78 9123", ('111 BCD Cir', 'Some City', 'Some State', 500091)
     ),
     (3, "Nick", "Junior", 750.0, 
      "United Kingdom", ['nick@junior.com', 'njunior@companyx.com'], 
      {"Home": "+44 111 111 1111", "Office": "+44 222 222 2222"}, 
      "222 33 4444", ('222 Giant Cly', 'UK City', 'UK Province', None)
     ),
     (4, "Bill", "Gomes", 1500.0, 
      "Australia", ['bill@gomes.com', 'bgomes@companyx.com'], 
      {"Home": "+61 987 654 3210", "Office": "+61 876 543 2109"}, 
      "789 12 6118", None
     ),
     (5, 'Harry', 'Potter', 1800.0,
      'United States', None, None, None, None
     )
]

In [ ]:
employees_df = spark.createDataFrame(
    employees,
    schema="""employee_id INT, employee_first_name STRING, employee_last_name STRING,
        employee_salary FLOAT, employee_nationality STRING, employee_email_ids ARRAY<STRING>,
        employee_phone_numbers MAP<STRING, STRING>, employee_ssn STRING,
        employee_address STRUCT<street: STRING, city: STRING, state: STRING, postal_code: INT>
    """
)

In [ ]:
employees_df.printSchema()

In [ ]:
employees_df.show(truncate=False)

In [ ]:
employees_df.select('employee_email_ids').show(truncate=False)

* We can use `explode` function to explode an array into multiple rows. Let us get employee id with email ids exploded into multiple rows.
* If the array column for the record on which `explode` is used have null values, then that record will be discarded.

In [ ]:
employees_df.count()

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
employees_df.select('employee_id', explode('employee_email_ids').alias('employee_id')).show(truncate=False)

In [ ]:
employees_df.select('employee_id', explode('employee_email_ids')).count()

* We can use `explode_outer` to get the rows where array type column such as phone_numbers is null.

In [ ]:
from pyspark.sql.functions import explode_outer

In [ ]:
employees_df.select('employee_id', explode_outer('employee_email_ids').alias('employee_id')).show(truncate=False)

In [ ]:
employees_df.select('employee_id', explode_outer('employee_email_ids')).count()

* We can use `concat_ws` on top of email ids to convert array into delimited string.

In [ ]:
from pyspark.sql.functions import concat_ws

In [ ]:
employees_df. \
    select('employee_id', concat_ws(', ', 'employee_email_ids').alias('employee_email_ids')). \
    show(truncate=False)

* We can convert delimited string into array using `split` function. We are recreating employees dataframe with email ids as string where multiple email ids are delimited using `,`.

In [ ]:
employees = [
     (2, "Henry", "Ford", 1250.0, 
      "India", 'henry@ford.com, hford@companyx.com', 
      {"Home": "+91 234 567 8901", "Office": "+91 345 678 9012"}, 
      "456 78 9123", ('111 BCD Cir', 'Some City', 'Some State', 500091)
     ),
     (3, "Nick", "Junior", 750.0, 
      "United Kingdom", 'nick@junior.com, njunior@companyx.com', 
      {"Home": "+44 111 111 1111", "Office": "+44 222 222 2222"}, 
      "222 33 4444", ('222 Giant Cly', 'UK City', 'UK Province', None)
     ),
     (4, "Bill", "Gomes", 1500.0, 
      "Australia", 'bill@gomes.com, bgomes@companyx.com', 
      {"Home": "+61 987 654 3210", "Office": "+61 876 543 2109"}, 
      "789 12 6118", None
     ),
     (5, 'Harry', 'Potter', 1800.0,
      'United States', None, None, None, None
     )
]

In [ ]:
employees_df = spark.createDataFrame(
    employees,
    schema="""employee_id INT, employee_first_name STRING, employee_last_name STRING,
        employee_salary FLOAT, employee_nationality STRING, employee_email_ids STRING,
        employee_phone_numbers MAP<STRING, STRING>, employee_ssn STRING,
        employee_address STRUCT<street: STRING, city: STRING, state: STRING, postal_code: INT>
    """
)

In [ ]:
employees_df. \
    select('employee_id', 'employee_email_ids'). \
    show(truncate=False)

In [ ]:
from pyspark.sql.functions import split

In [ ]:
employees_df. \
    select('employee_id', split('employee_email_ids', ', ').alias('employee_email_ids')). \
    show(truncate=False)

In [ ]:
employees_df. \
    select('employee_id', split('employee_email_ids', ', ').alias('employee_email_ids')). \
    printSchema()

In [ ]:
employees_df. \
    select('employee_id', split('employee_email_ids', ', ').alias('employee_email_ids')). \
    show()

In [ ]:
employees_df. \
    select('employee_id', explode_outer(split('employee_email_ids', ', ')).alias('employee_email_id')). \
    printSchema()

In [ ]:
employees_df. \
    select('employee_id', explode_outer(split('employee_email_ids', ', ')).alias('employee_email_id')). \
    show()